In [ ]:
!pip install -q --upgrade wandb
!pip install sentence_transformers
!pip install flair

In [ ]:
import pandas as pd
import os

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Now, you can access files in your Google Drive
# For example, to read a text file
INPUT_DIR = 'data'

TRAIN_CSV = os.path.join(INPUT_DIR, 'train-table-mediumparser.csv')
TEST_CSV = os.path.join(INPUT_DIR, 'eval-table-mediumparser.csv')

# Train data
df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

Mounted at /content/drive


# Style / Semantics

In [ ]:
from sentence_transformers import SentenceTransformer

sbert_path = os.path.join("weights", "Siamese/fine-tuned-S-BERT")
sbert = SentenceTransformer(sbert_path)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
embeddings1 = []
embeddings2 = []
arr1 = df['Paragraphs1'].tolist()
arr2 = df['Paragraphs2'].tolist()

for s in arr1:
    s = sbert.encode(s)
    embeddings1.append(s)
    if len(embeddings1) % 1000 == 0:
      print(len(embeddings1))

for s in arr2:
    s = sbert.encode(s)
    embeddings2.append(s)
    if len(embeddings1) % 1000 == 0:
      print(len(embeddings1))

df['Embeddings1'] = embeddings1
df['Embeddings2'] = embeddings2

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [ ]:
embeddings1 = []
embeddings2 = []
arr1 = test_df['Paragraphs1'].tolist()
arr2 = test_df['Paragraphs2'].tolist()

for s in arr1:
    s = sbert.encode(s)
    embeddings1.append(s)
    if len(embeddings1) % 1000 == 0:
      print(len(embeddings1))

for s in arr2:
    s = sbert.encode(s)
    embeddings2.append(s)
    if len(embeddings2) % 1000 == 0:
      print(len(embeddings2))

test_df['Embeddings1'] = embeddings1
test_df['Embeddings2'] = embeddings2

1000
2000
3000
4000
1000
2000
3000
4000


In [ ]:
import pickle
import sys
sty_path = "stylometry_extractor.pkl"

from redditparser import Reddit_Parser

In [ ]:


class StylometryExtractor():
    def __init__(self, corpus):
        self.corpus = corpus
        self.embedding_vocab = api.load("glove-wiki-gigaword-300")
        self.top_100_common_words = None
        self.top_10_common_adjectives = None
        self.top_10_common_conjunctions = None
        self.top_10_common_interrogatives = None
        self.top_10_common_nouns = None
        self.top_10_common_verbs = None
        self.top_30_common_three_grams = None
        self.top_30_common_five_grams = None
        self.top_100_common_oov_embeddings = None
        self.top_100_common_tri_grams = None
        self.top_100_common_five_grams = None

    def is_consonant(self, char):
        consonant_pattern = re.compile(r'[bcdfghjklmnpqrstvwxyz]', re.IGNORECASE)
        return bool(consonant_pattern.match(char))

    def is_vowel(self, char):
        vowel_pattern = re.compile(r'[aeiou]', re.IGNORECASE)
        return bool(vowel_pattern.match(char))

    def preprocess_corpus(self):

        corpus = self.corpus
        print("Saved corpus variable")

        # Preprocessing step

        corpus_lower = corpus.lower()
        print("lowered")
        tokens = word_tokenize(corpus_lower)
        print("tokenized")
        tagged_tokens = pos_tag(tokens)
        print("pos_tagged")
        freq_dist = FreqDist(tagged_tokens)
        freq_dist_no_tag = FreqDist(tokens)
        print("general freq dist done")
        vocabulary = freq_dist_no_tag.keys()
        print("vocabulary initialized")
        adjectives = [word for word, tag in tagged_tokens if tag.startswith('JJ')]
        print("adjectives done")
        conjunctions = [word for word, tag in tagged_tokens if tag.startswith('CC')]
        print("conjunctions done")
        interrogatives = [word for word, tag in tagged_tokens if tag in ['WP', 'WRB']]
        print("interrogatives done")
        nouns = [word for word, tag in tagged_tokens if tag in ['NN', 'NNS', 'NNPS', 'NNP']]
        print("nouns done")
        verbs = [word for word, tag in tagged_tokens if tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]
        print("verbs done")
        three_grams = list(ngrams(tokens, 3))
        print("3grams done")
        five_grams = list(ngrams(tokens, 5))
        print("5grams done")
        oov_embeddings = [word for word in vocabulary if word not in self.embedding_vocab]
        print("oov done")

        ''''''
        freq_dist_adjectives = FreqDist(adjectives)
        print("collected freq_dists for adjectives")
        freq_dist_conjunctions = FreqDist(conjunctions)
        print("collected freq_dists for conjunctions")
        freq_dist_interrogatives = FreqDist(interrogatives)
        print("collected freq_dists for interrogatives")
        freq_dist_nouns = FreqDist(nouns)
        print("collected freq_dists for nouns")
        freq_dist_verbs = FreqDist(verbs)
        print("collected freq_dists for verbs")
        freq_dist_oov_embeddings = FreqDist(oov_embeddings)
        print("collected freq_dists for oovs")
        freq_dist_three_grams = FreqDist(three_grams)
        print("collected freq_dists for 3grams")
        freq_dist_five_grams = FreqDist(five_grams)
        print("collected freq_dists for 5grams")

        ''''''
        print("starting collecting most commons")
        self.top_100_common_words = [x[0] for x in freq_dist.most_common(100)]
        self.top_10_common_adjectives = [x[0] for x in freq_dist_adjectives.most_common(10)]
        self.top_10_common_conjunctions = [x[0] for x in freq_dist_conjunctions.most_common(10)]
        self.top_10_common_interrogatives = [x[0] for x in freq_dist_interrogatives.most_common(10)]
        self.top_10_common_nouns = [x[0] for x in freq_dist_nouns.most_common(10)]
        self.top_10_common_verbs = [x[0] for x in freq_dist_verbs.most_common(10)]
        self.top_30_common_three_grams = [x[0] for x in freq_dist_three_grams.most_common(30)]
        self.top_30_common_five_grams = [x[0] for x in freq_dist_five_grams.most_common(30)]
        self.top_100_common_oov_embeddings = [x[0] for x in freq_dist_oov_embeddings.most_common(100)]
        self.top_100_common_tri_grams = [x[0] for x in freq_dist_five_grams.most_common(100)]
        self.top_100_common_five_grams = [x[0] for x in freq_dist_three_grams.most_common(100)]

        # Update init to save info about the corpus

    def get_values(self, value):

        if value == 'words':
            return self.top_100_common_words
        elif value == 'adjectives':
            return self.top_10_common_adjectives
        elif value == 'conjunctions':
            return self.top_10_common_conjunctions
        elif value == 'interrogatives':
            return self.top_10_common_interrogatives
        elif value == 'nouns':
            return self.top_10_common_nouns
        elif value == 'verbs':
            return self.top_10_common_verbs
        elif value == 'three_grams':
            return self.top_30_common_three_grams
        elif value == 'five_grams':
            return self.top_30_common_five_grams
        elif value == 'oov':
            return self.top_100_common_oov_embeddings
        else:
            ValueError('Set value')

    def stylometry_extractor(self, paragraph, character_level=True, word_level=True, sentence_level=True):

        # Some init
        F = np.array([])
        F = F.reshape(-1)
        F_chars = np.zeros(104)
        F_words = np.zeros(60)
        F_sents = np.zeros(220)

        epsilon = 1e-10

        # Preprocess paragraph
        sentences = sent_tokenize(paragraph)
        tokens = word_tokenize(paragraph)
        paragraph_lower = paragraph.lower()
        tokens_lower = word_tokenize(paragraph_lower)
        pos_tagged_tokens = pos_tag(tokens)
        parts_of_speech = [x[1] for x in pos_tagged_tokens]

        # Set regexs

        punctuation = r'[\W_]+'
        other_things = r'[^a-zA-Z0-9\s\\p{P}]'
        words = len(tokens)

        # Character level

        chars_list = list(paragraph_lower)
        chars = len(list(paragraph_lower))
        alphas = len([char for char in chars_list if char.isalpha()])
        uppers = len([char for char in list(paragraph) if char.isupper()])
        digits = len([char for char in chars_list if char.isdigit()])
        whitespaces = len([char for char in chars_list if char == ' '])
        vowels = len([char for char in paragraph if char.lower() in 'aeiou'])
        char_two_grams = len([''.join(gram) for gram in list(ngrams(chars_list, 2))])
        consonant_vowel_twograms = len([''.join(gram) for gram in ngrams(chars_list, 2) if re.match(r'[bcdfghjklmnpqrstvwxyz][aeiou]', ''.join(gram))])
        vowel_consonant_twograms = len([''.join(gram) for gram in ngrams(chars_list, 2) if re.match(r'[aeiou][bcdfghjklmnpqrstvwxyz]', ''.join(gram))])
        consonant_consonant_twograms = len([''.join(gram) for gram in ngrams(chars_list, 2) if re.match(r'[bcdfghjklmnpqrstvwxyz][bcdfghjklmnpqrstvwxyz]', ''.join(gram))])
        vowel_vowel_twograms = len([''.join(gram) for gram in ngrams(chars_list, 2) if re.match(r'[aeiou][aeiou]', ''.join(gram))])

        # Word level

        word_three_grams = list(ngrams(tokens, 3))
        word_five_grams = list(ngrams(tokens, 5))

        # Sentence level

        puncts = len(re.findall(punctuation, paragraph))
        others = len(re.findall(other_things, paragraph))
        pp = len([pos_tag for pos_tag in parts_of_speech if pos_tag.startswith('PRP')])
        adj = len([pos_tag for pos_tag in parts_of_speech if pos_tag.startswith('JJ')])
        conj = len([pos_tag for pos_tag in parts_of_speech if pos_tag.startswith('CC')])
        aux = len([pos_tag for pos_tag in parts_of_speech if pos_tag.startswith('MD')])
        interr = len([pos_tag for pos_tag in parts_of_speech if pos_tag in ['WP', 'WRB']])
        nouns = len([pos_tag for pos_tag in parts_of_speech if pos_tag in ['NN', 'NNS', 'NNPS', 'NNP']])
        verbs = len([pos_tag for pos_tag in parts_of_speech if pos_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']])
        dets = len([pos_tag for pos_tag in parts_of_speech if pos_tag.startswith('DT')])
        articles = len([token for token in tokens_lower if token in ["a", "an", "the"]])
        sents = len(nltk.sent_tokenize(paragraph))
        top100_common_words = [word for word, _ in self.top_100_common_words]

        # Build feature vector

        freq_cons_0 = ["t", "n", "s", "r", "h"]
        count_freq_cons_0 = sum(paragraph_lower.count(char) for char in freq_cons_0)
        freq_cons_1 = ["l", "d", "c", "p", "f"]
        count_freq_cons_1 = sum(paragraph_lower.count(char) for char in freq_cons_1)
        freq_cons_2 = ["m", "w", "y", "b", "g"]
        count_freq_cons_2 = sum(paragraph_lower.count(char) for char in freq_cons_2)
        freq_cons_3 = ["j", "k", "q", "v", "x", "z"]
        count_freq_cons_3 = sum(paragraph_lower.count(char) for char in freq_cons_3)
        to_be_verbs = [r'\b(am)\b', r'\b(are)\b', r'\b(be)\b', r'\b(been)\b', r'\b(being)\b', r'\b(is)\b', r'\b(was)\b', r'\b(were)\b']
        count_to_be_verbs = sum(paragraph_lower.count(word) for word in to_be_verbs)

        def count_missed_uppercase(tokens):
            missed_uppercase_count = 0

            for i in range(len(tokens) - 1):
                if tokens[i] == '.' and tokens[i+1][0].islower():
                    missed_uppercase_count += 1

            return missed_uppercase_count

        def count_missing_periods(sentences):
            missing_period_count = 0

            for sentence in sentences:
                if sentence[-1] != '.':
                    missing_period_count += 1

            return missing_period_count

        if character_level:
            F_chars = [
                alphas/(chars + epsilon),
                uppers/(chars + epsilon),
                digits/(chars + epsilon),
                whitespaces/(chars + epsilon),
                vowels/(chars + epsilon),
                paragraph_lower.count("a")/(vowels + epsilon),
                paragraph_lower.count("e")/(vowels + epsilon),
                paragraph_lower.count("i")/(vowels + epsilon),
                paragraph_lower.count("o")/(vowels + epsilon),
                paragraph_lower.count("u")/(vowels + epsilon),
                paragraph_lower.count("a")/(chars + epsilon),
                count_freq_cons_0/(alphas + epsilon),
                count_freq_cons_1/(alphas + epsilon),
                count_freq_cons_2/(alphas + epsilon),
                count_freq_cons_3/(alphas + epsilon),
                paragraph_lower.count("t")/(count_freq_cons_0 + epsilon),
                paragraph_lower.count("n")/(count_freq_cons_0 + epsilon),
                paragraph_lower.count("s")/(count_freq_cons_0 + epsilon),
                paragraph_lower.count("r")/(count_freq_cons_0 + epsilon),
                paragraph_lower.count("h")/(count_freq_cons_0 + epsilon),
                paragraph_lower.count("l")/(count_freq_cons_1 + epsilon),
                paragraph_lower.count("d")/(count_freq_cons_1 + epsilon),
                paragraph_lower.count("c")/(count_freq_cons_1 + epsilon),
                paragraph_lower.count("p")/(count_freq_cons_1 + epsilon),
                paragraph_lower.count("f")/(count_freq_cons_1 + epsilon),
                paragraph_lower.count("m")/(count_freq_cons_2 + epsilon),
                paragraph_lower.count("w")/(count_freq_cons_2 + epsilon),
                paragraph_lower.count("y")/(count_freq_cons_2 + epsilon),
                paragraph_lower.count("b")/(count_freq_cons_2 + epsilon),
                paragraph_lower.count("g")/(count_freq_cons_2 + epsilon),
                paragraph_lower.count("j")/(count_freq_cons_3 + epsilon),
                paragraph_lower.count("k")/(count_freq_cons_3 + epsilon),
                paragraph_lower.count("q")/(count_freq_cons_3 + epsilon),
                paragraph_lower.count("v")/(count_freq_cons_3 + epsilon),
                paragraph_lower.count("x")/(count_freq_cons_3 + epsilon),
                paragraph_lower.count("z")/(count_freq_cons_3 + epsilon),
                consonant_consonant_twograms/(char_two_grams + epsilon),
                vowel_consonant_twograms/(char_two_grams + epsilon),
                consonant_vowel_twograms/(char_two_grams + epsilon),
                vowel_vowel_twograms/(char_two_grams + epsilon),
                paragraph_lower.count("st")/(consonant_consonant_twograms + epsilon),
                paragraph_lower.count("nd")/(consonant_consonant_twograms + epsilon),
                paragraph_lower.count("th")/(consonant_consonant_twograms + epsilon),
                paragraph_lower.count("an")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("in")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("er")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("es")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("on")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("at")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("en")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("or")/(vowel_consonant_twograms + epsilon),
                paragraph_lower.count("he")/(consonant_vowel_twograms + epsilon),
                paragraph_lower.count("re")/(consonant_vowel_twograms + epsilon),
                paragraph_lower.count("ti")/(consonant_vowel_twograms + epsilon),
                paragraph_lower.count("ea")/(vowel_vowel_twograms + epsilon),
                sum(1 for i in range(len(paragraph_lower) - 1) if paragraph_lower[i].isalpha() and paragraph_lower[i] == paragraph_lower[i + 1])/(char_two_grams + epsilon),
                paragraph_lower.count("a"),
                paragraph_lower.count("b"),
                paragraph_lower.count("c"),
                paragraph_lower.count("d"),
                paragraph_lower.count("e"),
                paragraph_lower.count("f"),
                paragraph_lower.count("g"),
                paragraph_lower.count("h"),
                paragraph_lower.count("i"),
                paragraph_lower.count("j"),
                paragraph_lower.count("k"),
                paragraph_lower.count("l"),
                paragraph_lower.count("m"),
                paragraph_lower.count("n"),
                paragraph_lower.count("o"),
                paragraph_lower.count("p"),
                paragraph_lower.count("q"),
                paragraph_lower.count("r"),
                paragraph_lower.count("s"),
                paragraph_lower.count("t"),
                paragraph_lower.count("u"),
                paragraph_lower.count("v"),
                paragraph_lower.count("w"),
                paragraph_lower.count("x"),
                paragraph_lower.count("y"),
                paragraph_lower.count("z"),
                paragraph_lower.count("~"),
                paragraph_lower.count("@"),
                paragraph_lower.count("#"),
                paragraph_lower.count("$"),
                paragraph_lower.count("%"),
                paragraph_lower.count("^"),
                paragraph_lower.count("&"),
                paragraph_lower.count("*"),
                paragraph_lower.count("-"),
                paragraph_lower.count("="),
                paragraph_lower.count("+"),
                paragraph_lower.count(">"),
                paragraph_lower.count("<"),
                paragraph_lower.count("["),
                paragraph_lower.count("]"),
                paragraph_lower.count("{"),
                paragraph_lower.count("}"),
                paragraph_lower.count("/"),
                paragraph_lower.count("\\"),
                paragraph_lower.count("|"),
            ]
            F = np.concatenate((F, F_chars))

        if word_level:
            F_words = [
                len([token for token in tokens if len(token) == 1])/(words + epsilon),
                len([token for token in tokens if len(token) == 2])/(words + epsilon),
                len([token for token in tokens if len(token) == 3])/(words + epsilon),
                len([token for token in tokens if len(token) == 4])/(words + epsilon),
                len([token for token in tokens if len(token) == 5])/(words + epsilon),
                len([token for token in tokens if len(token) == 6])/(words + epsilon),
                len([token for token in tokens if len(token) == 7])/(words + epsilon),
                len([token for token in tokens if len(token) >= 8])/(words + epsilon),
                len([token for token in tokens if len(token) <= 3])/(words + epsilon),
                chars/(words + epsilon),
                len(set(tokens))/(words + epsilon),
                words,
                len([token for token in tokens if len(token) <= 3]),
                len([token for token in tokens if len(token) == 1]),
                len([token for token in tokens if len(token) == 2]),
                len([token for token in tokens if len(token) == 3]),
                len([token for token in tokens if len(token) == 4]),
                len([token for token in tokens if len(token) == 5]),
                len([token for token in tokens if len(token) == 6]),
                len([token for token in tokens if len(token) == 7]),
                len([token for token in tokens if len(token) == 8]),
                len([token for token in tokens if len(token) == 9]),
                len([token for token in tokens if len(token) == 10]),
                len([token for token in tokens if len(token) == 11]),
                len([token for token in tokens if len(token) == 12]),
                len([token for token in tokens if len(token) >= 13]),
                paragraph_lower.count(":)"),
                paragraph_lower.count(":("),
                paragraph_lower.count(r'\b(lol)\b'),
                paragraph_lower.count(";)"),
                paragraph_lower.count("..."),
                paragraph_lower.count("cmv"),
                paragraph_lower.count("eli5"),
                paragraph_lower.count("iirc"),
                paragraph_lower.count("imo"),
                paragraph_lower.count("imho"),
                paragraph_lower.count("irl"),
                paragraph_lower.count("mrw"),
                paragraph_lower.count("mfw"),
                paragraph_lower.count("nsfl"),
                paragraph_lower.count("nsfw"),
                paragraph_lower.count(r'\b(op)\b'),
                paragraph_lower.count(r'\b(oc)\b'),
                paragraph_lower.count("psa"),
                paragraph_lower.count("tldr"),
                paragraph_lower.count("tl;dr"),
                paragraph_lower.count(r'\b(til)\b'),
                paragraph_lower.count("wip"),
                paragraph_lower.count("ysk"),
                paragraph_lower.count("aka"),
                paragraph_lower.count("goat"),
                paragraph_lower.count(r'\b(ffs)\b'),
                paragraph_lower.count("fyi"),
                paragraph_lower.count("tbh"),
                paragraph_lower.count("ikr"),
                count_missed_uppercase(tokens),
                count_missing_periods(sentences),
                sum(1 for token in tokens if token in self.top_100_common_oov_embeddings)/(words + epsilon),
                sum(1 for trigram in word_three_grams if trigram in self.top_100_common_tri_grams)/(len(word_three_grams) + epsilon),
                sum(1 for fivegram in word_five_grams if fivegram in self.top_100_common_five_grams)/(len(word_five_grams) + epsilon),
            ]
            F = np.concatenate((F, F_words))

        if sentence_level:
            F_sents = [
                sents,
                puncts/chars,
                paragraph_lower.count('.')/(puncts + epsilon),
                paragraph_lower.count(',')/(puncts + epsilon),
                paragraph_lower.count('?')/(puncts + epsilon),
                paragraph_lower.count('!')/(puncts + epsilon),
                paragraph_lower.count(';')/(puncts + epsilon),
                paragraph_lower.count(':')/(puncts + epsilon),
                paragraph_lower.count('\'')/(puncts + epsilon),
                paragraph_lower.count('\"')/(puncts + epsilon),
                others/(chars + epsilon),
                digits/(others + epsilon),
                conj/(words + epsilon),
                interr/(words + epsilon),
                pp/(words + epsilon),
                nouns/(words + epsilon),
                verbs/(words + epsilon),
                adj/(words + epsilon),
                articles/(words + epsilon),
                articles/(adj + epsilon),
                dets/(words + epsilon),
                aux/(words + epsilon),
                aux/(verbs + epsilon),
                chars/(sents + epsilon),
                words/(sents + epsilon),
                paragraph_lower.count("can")/(aux + epsilon),
                paragraph_lower.count("did")/(aux + epsilon),
                paragraph_lower.count(r'\b(do)\b')/(aux + epsilon),
                paragraph_lower.count("does")/(aux + epsilon),
                paragraph_lower.count(r'\b(had)\b')/(aux + epsilon),
                paragraph_lower.count(r'\b(has)\b')/(aux + epsilon),
                paragraph_lower.count("have")/(aux + epsilon),
                paragraph_lower.count("could")/(aux + epsilon),
                paragraph_lower.count("should")/(aux + epsilon),
                paragraph_lower.count("would")/(aux + epsilon),
                paragraph_lower.count(r'\b(will)\b')/(aux + epsilon),
                count_to_be_verbs/(words + epsilon),
                count_to_be_verbs/(verbs + epsilon),
                paragraph_lower.count(r'\b(am)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(are)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(be)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(is)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(was)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(were)\b')/(count_to_be_verbs + epsilon),
                paragraph_lower.count(r'\b(the)\b')/(articles + epsilon),
                paragraph_lower.count(r'\b(a)\b')/(articles + epsilon),
                paragraph_lower.count(r'\b(an)\b')/(articles + epsilon),
                paragraph_lower.count(top100_common_words[0])/(words + epsilon),
                paragraph_lower.count(top100_common_words[1])/(words + epsilon),
                paragraph_lower.count(top100_common_words[2])/(words + epsilon),
                paragraph_lower.count(top100_common_words[3])/(words + epsilon),
                paragraph_lower.count(top100_common_words[4])/(words + epsilon),
                paragraph_lower.count(top100_common_words[5])/(words + epsilon),
                paragraph_lower.count(top100_common_words[6])/(words + epsilon),
                paragraph_lower.count(top100_common_words[7])/(words + epsilon),
                paragraph_lower.count(top100_common_words[8])/(words + epsilon),
                paragraph_lower.count(top100_common_words[9])/(words + epsilon),
                paragraph_lower.count(top100_common_words[10])/(words + epsilon),
                paragraph_lower.count(top100_common_words[11])/(words + epsilon),
                paragraph_lower.count(top100_common_words[12])/(words + epsilon),
                paragraph_lower.count(top100_common_words[13])/(words + epsilon),
                paragraph_lower.count(top100_common_words[14])/(words + epsilon),
                paragraph_lower.count(top100_common_words[15])/(words + epsilon),
                paragraph_lower.count(top100_common_words[16])/(words + epsilon),
                paragraph_lower.count(top100_common_words[17])/(words + epsilon),
                paragraph_lower.count(top100_common_words[18])/(words + epsilon),
                paragraph_lower.count(top100_common_words[19])/(words + epsilon),
                paragraph_lower.count(top100_common_words[20])/(words + epsilon),
                paragraph_lower.count(top100_common_words[21])/(words + epsilon),
                paragraph_lower.count(top100_common_words[22])/(words + epsilon),
                paragraph_lower.count(top100_common_words[23])/(words + epsilon),
                paragraph_lower.count(top100_common_words[24])/(words + epsilon),
                paragraph_lower.count(top100_common_words[25])/(words + epsilon),
                paragraph_lower.count(top100_common_words[26])/(words + epsilon),
                paragraph_lower.count(top100_common_words[27])/(words + epsilon),
                paragraph_lower.count(top100_common_words[28])/(words + epsilon),
                paragraph_lower.count(top100_common_words[29])/(words + epsilon),
                paragraph_lower.count(top100_common_words[30])/(words + epsilon),
                paragraph_lower.count(top100_common_words[31])/(words + epsilon),
                paragraph_lower.count(top100_common_words[32])/(words + epsilon),
                paragraph_lower.count(top100_common_words[33])/(words + epsilon),
                paragraph_lower.count(top100_common_words[34])/(words + epsilon),
                paragraph_lower.count(top100_common_words[35])/(words + epsilon),
                paragraph_lower.count(top100_common_words[36])/(words + epsilon),
                paragraph_lower.count(top100_common_words[37])/(words + epsilon),
                paragraph_lower.count(top100_common_words[38])/(words + epsilon),
                paragraph_lower.count(top100_common_words[39])/(words + epsilon),
                paragraph_lower.count(top100_common_words[40])/(words + epsilon),
                paragraph_lower.count(top100_common_words[41])/(words + epsilon),
                paragraph_lower.count(top100_common_words[42])/(words + epsilon),
                paragraph_lower.count(top100_common_words[43])/(words + epsilon),
                paragraph_lower.count(top100_common_words[44])/(words + epsilon),
                paragraph_lower.count(top100_common_words[45])/(words + epsilon),
                paragraph_lower.count(top100_common_words[46])/(words + epsilon),
                paragraph_lower.count(top100_common_words[47])/(words + epsilon),
                paragraph_lower.count(top100_common_words[48])/(words + epsilon),
                paragraph_lower.count(top100_common_words[49])/(words + epsilon),
                paragraph_lower.count(r'\b(a)\b'),
                paragraph_lower.count('about'),
                paragraph_lower.count('above'),
                paragraph_lower.count('after'),
                paragraph_lower.count(r'\b(all)\b'),
                paragraph_lower.count('although'),
                paragraph_lower.count(r'\b(am)\b'),
                paragraph_lower.count('among'),
                paragraph_lower.count(r'\b(an)\b'),
                paragraph_lower.count(r'\b(and)\b'),
                paragraph_lower.count('another'),
                paragraph_lower.count(r'\b(any)\b'),
                paragraph_lower.count('anybody'),
                paragraph_lower.count('anyone'),
                paragraph_lower.count('anything'),
                paragraph_lower.count(r'\b(are)\b'),
                paragraph_lower.count('around'),
                paragraph_lower.count(r'\b(as)\b'),
                paragraph_lower.count(r'\b(at)\b'),
                paragraph_lower.count(r'\b(be)\b'),
                paragraph_lower.count('because'),
                paragraph_lower.count('before'),
                paragraph_lower.count('behind'),
                paragraph_lower.count('below'),
                paragraph_lower.count('beside'),
                paragraph_lower.count('between'),
                paragraph_lower.count('both'),
                paragraph_lower.count(r'\b(but)\b'),
                paragraph_lower.count(r'\b(by)\b'),
                paragraph_lower.count(r'\b(can)\b'),
                paragraph_lower.count(r'\b(do)\b'),
                paragraph_lower.count('down'),
                paragraph_lower.count('each'),
                paragraph_lower.count('either'),
                paragraph_lower.count('enough'),
                paragraph_lower.count('every'),
                paragraph_lower.count('everybody'),
                paragraph_lower.count('everyone'),
                paragraph_lower.count('everything'),
                paragraph_lower.count('few'),
                paragraph_lower.count('following'),
                paragraph_lower.count(r'\b(for)\b'),
                paragraph_lower.count('from'),
                paragraph_lower.count('have'),
                paragraph_lower.count(r'\b(he)\b'),
                paragraph_lower.count(r'\b(her)\b'),
                paragraph_lower.count(r'\b(him)\b'),
                paragraph_lower.count(r'\b(i)\b'),
                paragraph_lower.count(r'\b(if)\b'),
                paragraph_lower.count(r'\b(in)\b'),
                paragraph_lower.count('including'),
                paragraph_lower.count('inside'),
                paragraph_lower.count(r'\b(into)\b'),
                paragraph_lower.count(r'\b(is)\b'),
                paragraph_lower.count(r'\b(it)\b'),
                paragraph_lower.count(r'\b(its)\b'),
                paragraph_lower.count('latter'),
                paragraph_lower.count(r'\b(less)\b'),
                paragraph_lower.count(r'\b(like)\b'),
                paragraph_lower.count('little'),
                paragraph_lower.count(r'\b(lots)\b'),
                paragraph_lower.count('many'),
                paragraph_lower.count(r'\b(me)\b'),
                paragraph_lower.count(r'\b(more)\b'),
                paragraph_lower.count(r'\b(most)\b'),
                paragraph_lower.count('much'),
                paragraph_lower.count(r'\b(my)\b'),
                paragraph_lower.count('need'),
                paragraph_lower.count('neither'),
                paragraph_lower.count(r'\b(no)\b'),
                paragraph_lower.count('nobody'),
                paragraph_lower.count(r'\b(none)\b'),
                paragraph_lower.count(r'\b(nor)\b'),
                paragraph_lower.count('nothing'),
                paragraph_lower.count(r'\b(of)\b'),
                paragraph_lower.count(r'\b(off)\b'),
                paragraph_lower.count(r'\b(on)\b'),
                paragraph_lower.count('once'),
                paragraph_lower.count(r'\b(one)\b'),
                paragraph_lower.count('onto'),
                paragraph_lower.count('opposite'),
                paragraph_lower.count(r'\b(or)\b'),
                paragraph_lower.count(r'\b(our)\b'),
                paragraph_lower.count('outside'),
                paragraph_lower.count('over'),
                paragraph_lower.count(r'\b(some)\b'),
                paragraph_lower.count('somebody'),
                paragraph_lower.count('someone'),
                paragraph_lower.count('something'),
                paragraph_lower.count('such'),
                paragraph_lower.count('than'),
                paragraph_lower.count('that'),
                paragraph_lower.count(r'\b(the)\b'),
                paragraph_lower.count(r'\b(their)\b'),
                paragraph_lower.count(r'\b(them)\b'),
                paragraph_lower.count(r'\b(these)\b'),
                paragraph_lower.count(r'\b(they)\b'),
                paragraph_lower.count(r'\b(this)\b'),
                paragraph_lower.count(r'\b(those)\b'),
                paragraph_lower.count('though'),
                paragraph_lower.count('through'),
                paragraph_lower.count(r'\b(till)\b'),
                paragraph_lower.count(r'\b(to)\b'),
                paragraph_lower.count('toward '),
                paragraph_lower.count('towards'),
                paragraph_lower.count('under'),
                paragraph_lower.count('unless'),
                paragraph_lower.count('whether'),
                paragraph_lower.count('which'),
                paragraph_lower.count('while'),
                paragraph_lower.count(r'\b(who)\b'),
                paragraph_lower.count('whoever'),
                paragraph_lower.count('whom'),
                paragraph_lower.count('whose'),
                paragraph_lower.count(r'\b(will)\b'),
                paragraph_lower.count(r'\b(with)\b'),
                paragraph_lower.count('within'),
                paragraph_lower.count('without'),
                paragraph_lower.count('worth'),
                paragraph_lower.count('would'),
                paragraph_lower.count(r'\b(yes)\b'),
                paragraph_lower.count(r'\b(you)\b'),
                paragraph_lower.count(r'\b(your)\b'),
                paragraph_lower.count(r'([^\w\s])\1')/(char_two_grams + epsilon),
            ]
            F = np.concatenate((F, F_sents))

        return F


In [ ]:
with open(sty_path, "rb") as f:
    parser = pickle.load(f)

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

stylometrics1 = []
stylometrics2 = []
arr1 = df['Paragraphs1'].tolist()
arr2 = df['Paragraphs2'].tolist()

for s in arr1:
    s = parser.stylometry_extractor(s)
    stylometrics1.append(s)
    print(len(stylometrics1))

for s in arr2:
    s = parser.stylometry_extractor(s)
    stylometrics2.append(s)
    print(len(stylometrics2))

df['Stylometrics1'] = stylometrics1
df['Stylometrics2'] = stylometrics2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Output streaming troncato alle ultime 5000 righe.
14016
14017
14018
14019
14020
14021
14022
14023
14024
14025
14026
14027
14028
14029
14030
14031
14032
14033
14034
14035
14036
14037
14038
14039
14040
14041
14042
14043
14044
14045
14046
14047
14048
14049
14050
14051
14052
14053
14054
14055
14056
14057
14058
14059
14060
14061
14062
14063
14064
14065
14066
14067
14068
14069
14070
14071
14072
14073
14074
14075
14076
14077
14078
14079
14080
14081
14082
14083
14084
14085
14086
14087
14088
14089
14090
14091
14092
14093
14094
14095
14096
14097
14098
14099
14100
14101
14102
14103
14104
14105
14106
14107
14108
14109
14110
14111
14112
14113
14114
14115
14116
14117
14118
14119
14120
14121
14122
14123
14124
14125
14126
14127
14128
14129
14130
14131
14132
14133
14134
14135
14136
14137
14138
14139
14140
14141
14142
14143
14144
14145
14146
14147
14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14

In [ ]:
stylometrics1 = []
stylometrics2 = []
arr1 = test_df['Paragraphs1'].tolist()
arr2 = test_df['Paragraphs2'].tolist()

for s in arr1:
    s = parser.stylometry_extractor(s)
    stylometrics1.append(s)
    print(len(stylometrics1))

for s in arr2:
    s = parser.stylometry_extractor(s)
    stylometrics2.append(s)
    print(len(stylometrics2))

test_df['Stylometrics1'] = stylometrics1
test_df['Stylometrics2'] = stylometrics2

Output streaming troncato alle ultime 5000 righe.
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3456
3457
3458
3459
3460


# Topic / Sentiment

In [ ]:
from gensim.models import LdaModel

lda_model = LdaModel.load("/weights/Siamese/Topicmodel/Medium/Topic_model")

In [ ]:
arr = df['Paragraphs1'].tolist() + df['Paragraphs2'].tolist()
test_arr = test_df['Paragraphs1'].tolist() + test_df['Paragraphs2'].tolist()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import string
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

texts = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in set(arr)]
test_texts = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in set(test_arr)]

df.dropna()
test_df.dropna()

test_arr1 = test_df['Paragraphs1'].tolist()
test_arr2 = test_df['Paragraphs2'].tolist()

arr1 = df['Paragraphs1'].tolist()
arr2 = df['Paragraphs2'].tolist()
new_texts1 = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in arr1]
new_texts2 = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in arr2]

test_new_texts1 = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in test_arr1]
test_new_texts2 = [[word for word in word_tokenize(document) if word not in stop_words and word not in punctuation] for document in test_arr2]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in new_texts1]
topic_distributions = [lda_model.get_document_topics(sentence) for sentence in corpus]

In [ ]:
import numpy as np

padded_distributions = []
for d in topic_distributions:
  existing_values = dict(d)
  padded_list = [(n, existing_values.get(n, 0)) for n in range(100)]
  d = padded_list
  padded_distributions.append(d)

print(len(padded_distributions))
print(padded_distributions[0])

19015
[(0, 0), (1, 0.037162565), (2, 0), (3, 0), (4, 0.0327875), (5, 0), (6, 0), (7, 0), (8, 0.03477139), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0.033704434), (18, 0.06525043), (19, 0), (20, 0), (21, 0.065254025), (22, 0.03868408), (23, 0), (24, 0), (25, 0.043956313), (26, 0.038499396), (27, 0.03265307), (28, 0.032922577), (29, 0.046321042), (30, 0), (31, 0), (32, 0), (33, 0.06524588), (34, 0), (35, 0.032275084), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0.039178215), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0.06524005), (46, 0.040249493), (47, 0.039411925), (48, 0), (49, 0), (50, 0), (51, 0.032840997), (52, 0), (53, 0), (54, 0), (55, 0.08973041), (56, 0), (57, 0), (58, 0), (59, 0.032852285), (60, 0), (61, 0), (62, 0), (63, 0), (64, 0.038792476), (65, 0), (66, 0), (67, 0), (68, 0), (69, 0), (70, 0), (71, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (78, 0), (79, 0), (80, 0), (81, 0), (82, 0), (83, 0), (84, 0), (85, 0), (86, 0), (87, 0), 

In [ ]:
vectors = []

# Iterate through each item in padded_distributions
for item in padded_distributions:
    # Extract the first values from the tuples in the item
    vec = np.zeros(100)
    values = [value for _, value in item]
    for index, value in enumerate(values):
      vec[index] += value
    vectors.append(vec)
    # Append the encoded keys to the list

print(len(vectors))
print(len(df))

df['Topics1'] = vectors


19015
19015


In [ ]:
corpus = [dictionary.doc2bow(text) for text in new_texts2]
topic_distributions = [lda_model.get_document_topics(sentence) for sentence in corpus]

In [ ]:
padded_distributions = []

for d in topic_distributions:
  existing_values = dict(d)
  padded_list = [(n, existing_values.get(n, 0)) for n in range(100)]
  d = padded_list
  padded_distributions.append(d)

print(padded_distributions[0])

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0.10143165), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0.10145354), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0.30148998), (37, 0), (38, 0), (39, 0.101343416), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0.101449415), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (51, 0), (52, 0), (53, 0), (54, 0), (55, 0), (56, 0), (57, 0), (58, 0.10136153), (59, 0), (60, 0), (61, 0), (62, 0), (63, 0), (64, 0), (65, 0), (66, 0.101446785), (67, 0), (68, 0), (69, 0), (70, 0), (71, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (78, 0), (79, 0), (80, 0), (81, 0), (82, 0), (83, 0), (84, 0), (85, 0), (86, 0), (87, 0), (88, 0), (89, 0), (90, 0), (91, 0), (92, 0), (93, 0), (94, 0), (95, 0), (96, 0), (97, 0), (98, 0), (99, 0)]


In [ ]:
vectors = []

# Iterate through each item in padded_distributions
for item in padded_distributions:
    # Extract the first values from the tuples in the item
    vec = np.zeros(100)
    values = [value for _, value in item]
    for index, value in enumerate(values):
      vec[index] += value
    vectors.append(vec)
    # Append the encoded keys to the list

df['Topics2'] = vectors

In [ ]:
dictionary = corpora.Dictionary(test_texts)

corpus = [dictionary.doc2bow(text) for text in test_new_texts1]
topic_distributions = [lda_model.get_document_topics(sentence) for sentence in corpus]

padded_distributions = []
for d in topic_distributions:
  existing_values = dict(d)
  padded_list = [(n, existing_values.get(n, 0)) for n in range(100)]
  d = padded_list
  padded_distributions.append(d)

vectors = []

# Iterate through each item in padded_distributions
for item in padded_distributions:
    # Extract the first values from the tuples in the item
    vec = np.zeros(100)
    values = [value for _, value in item]
    for index, value in enumerate(values):
      vec[index] += value
    vectors.append(vec)
    # Append the encoded keys to the list

print(len(vectors))
print(len(df))

test_df['Topics1'] = vectors

4135
19015


In [ ]:
corpus = [dictionary.doc2bow(text) for text in test_new_texts2]
topic_distributions = [lda_model.get_document_topics(sentence) for sentence in corpus]

padded_distributions = []
for d in topic_distributions:
  existing_values = dict(d)
  padded_list = [(n, existing_values.get(n, 0)) for n in range(100)]
  d = padded_list
  padded_distributions.append(d)

vectors = []

# Iterate through each item in padded_distributions
for item in padded_distributions:
    # Extract the first values from the tuples in the item
    vec = np.zeros(100)
    values = [value for _, value in item]
    for index, value in enumerate(values):
      vec[index] += value
    vectors.append(vec)
    # Append the encoded keys to the list

print(len(vectors))
print(len(df))

test_df['Topics2'] = vectors

4135
19015


In [ ]:
!pip install --upgrade urllib3


from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flair 0.13.1 requires urllib3<2.0.0,>=1.0.0, but you have urllib3 2.2.1 which is incompatible.
2024-06-08 11:00:50,863 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpdkyh0a3u


100%|██████████| 253M/253M [00:06<00:00, 38.1MB/s]

2024-06-08 11:00:57,963 copying /tmp/tmpdkyh0a3u to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2024-06-08 11:00:58,786 removing temp file /tmp/tmpdkyh0a3u


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
arr1 = df['Paragraphs1'].tolist()

truncated_sentences = []
for sentence in arr1:
    # Split the sentence into tokens
    tokens = word_tokenize(sentence)

    if len(tokens) > 450:
      print("long")

    # Truncate to 512 tokens
    truncated_tokens = tokens[:450]

    # Join the truncated tokens back into a sentence
    truncated_sentence = ' '.join(truncated_tokens)

    # Add the truncated sentence to the list
    truncated_sentences.append(truncated_sentence)

arr1 = truncated_sentences

arr2 = df['Paragraphs2'].tolist()

truncated_sentences = []

for sentence in arr2:

    # Split the sentence into tokens
    tokens = word_tokenize(sentence)

    if len(tokens) > 450:
      print("long")

    # Truncate to 512 tokens
    truncated_tokens = tokens[:450]

    # Join the truncated tokens back into a sentence
    truncated_sentence = ' '.join(truncated_tokens)

    # Add the truncated sentence to the list
    truncated_sentences.append(truncated_sentence)

arr2 = truncated_sentences

long
long
long
long


In [ ]:
test_arr1 = test_df['Paragraphs1'].tolist()

truncated_sentences = []
for sentence in test_arr1:
    # Split the sentence into tokens
    tokens = word_tokenize(sentence)

    if len(tokens) > 450:
      print("long")

    # Truncate to 512 tokens
    truncated_tokens = tokens[:450]

    # Join the truncated tokens back into a sentence
    truncated_sentence = ' '.join(truncated_tokens)

    # Add the truncated sentence to the list
    truncated_sentences.append(truncated_sentence)

test_arr1 = truncated_sentences

test_arr2 = test_df['Paragraphs2'].tolist()

truncated_sentences = []

for sentence in test_arr2:

    # Split the sentence into tokens
    tokens = word_tokenize(sentence)

    if len(tokens) > 450:
      print("long")

    # Truncate to 512 tokens
    truncated_tokens = tokens[:450]

    # Join the truncated tokens back into a sentence
    truncated_sentence = ' '.join(truncated_tokens)

    # Add the truncated sentence to the list
    truncated_sentences.append(truncated_sentence)

test_arr2 = truncated_sentences

In [ ]:
def value(paragraph):
        sentence = Sentence(paragraph)
        classifier.predict(sentence)
        if sentence.labels[0].value == 'POSITIVE':
            return sentence.labels[0].score*10
        elif sentence.labels[0].value == 'NEGATIVE':
            return - sentence.labels[0].score*10

sentiments1 = []
sentiments2 = []
test_sentiments1 = []
test_sentiments2 = []

for s in arr1:
  s = value(s)
  sentiments1.append(s)
  print(len(sentiments1))

for s in arr2:
  s = value(s)
  sentiments2.append(s)
  print(len(sentiments2))

for s in test_arr1:
  s = value(s)
  test_sentiments1.append(s)
  print(len(test_sentiments1))

for s in test_arr2:
  s = value(s)
  test_sentiments2.append(s)
  print(len(test_sentiments2))


Output streaming troncato alle ultime 5000 righe.
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3456
3457
3458
3459
3460


In [ ]:
def min_max_scaling_with_index(lst, ref_lst):
    # Separate positive and negative values from the reference list
    positive_values = [x for x in ref_lst if x >= 0]
    negative_values = [x for x in ref_lst if x < 0]

    # Calculate mean and standard deviation for positive and negative values in the reference list
    mean_positive = np.mean(positive_values)
    std_positive = np.std(positive_values)
    mean_negative = np.mean(negative_values)
    std_negative = np.std(negative_values)

    # Scale the values in lst based on the reference list scaling parameters
    scaled_lst = []
    for value in lst:
        if value >= 0:
            if std_positive != 0:
                scaled_value = (value - mean_positive) / std_positive * 10
            else:
                scaled_value = 0  # Avoid division by zero if std is zero
        else:
            if std_negative != 0:
                scaled_value = (value - mean_negative) / std_negative * 10
            else:
                scaled_value = 0  # Avoid division by zero if std is zero
        scaled_lst.append(scaled_value)

    return scaled_lst

df['Sentiments1'] = min_max_scaling_with_index(sentiments1, sentiments1+sentiments2)
df['Sentiments2'] = min_max_scaling_with_index(sentiments2, sentiments1+sentiments2)

test_df['Sentiments1'] = min_max_scaling_with_index(test_sentiments1, test_sentiments1+test_sentiments2)
test_df['Sentiments2'] = min_max_scaling_with_index(test_sentiments2, test_sentiments1+test_sentiments2)

In [ ]:
df['Sentiments1'] = df['Sentiments1'].apply(lambda x: [x])
df['Sentiments2'] = df['Sentiments2'].apply(lambda x: [x])
test_df['Sentiments1'] = test_df['Sentiments1'].apply(lambda x: [x])
test_df['Sentiments2'] = test_df['Sentiments2'].apply(lambda x: [x])

# Model

In [ ]:
df.head()

,Paragraphs1,Paragraphs2,Truth_changes,file_number,Embeddings1,Embeddings2,Stylometrics1,Stylometrics2,Topics1,Topics2,Sentiments1,Sentiments2
0,I'm not gonna watch the video. I gotta keep my...,Maybe police agencies should be not federalize...,1,1,"[-0.03478252, 0.032403965, -0.0037362052, -0.0...","[0.036603045, -0.0648263, -0.009828456, 0.0084...","[0.7915309446251494, 0.019543973941361713, 0.0...","[0.7828571428566956, 0.022857142857129798, 0.0...","[0.0, 0.03716256469488144, 0.0, 0.0, 0.0327875...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10143165...",[-3.6174897014011025],[7.292997105151773]
1,Maybe police agencies should be not federalize...,Actual combat soldiers have much more discipli...,0,1,"[0.036603045, -0.0648263, -0.009828456, 0.0084...","[0.009029375, -0.02791714, -0.0065503856, -0.0...","[0.7828571428566956, 0.022857142857129798, 0.0...","[0.8006644518269765, 0.03322259136211521, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10143160...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0453801304101...",[7.292997105151773],[-4.694274813684402]
2,Actual combat soldiers have much more discipli...,"Police departments attract brutes, do nothing ...",1,1,"[0.009029375, -0.02791714, -0.0065503856, -0.0...","[0.021286048, -0.008837418, -0.04957801, -0.02...","[0.8006644518269765, 0.03322259136211521, 0.0,...","[0.8124999999996827, 0.031249999999987794, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0453795157372...","[0.0, 0.0, 0.06157473847270012, 0.0, 0.0567295...",[-4.694274813684402],[-4.3467248760075075]
3,"That's false, there are several factions invol...",Houthi Rebels representing Houthis that have b...,1,10,"[0.0330065, 0.012140937, -0.009300589, -0.0254...","[-0.017771857, 0.11146448, -0.03416922, -0.047...","[0.8141809290951555, 0.06845965770169475, 0.0,...","[0.8523809523805466, 0.028571428571414967, 0.0...","[0.0, 0.0, 0.0, 0.07833819836378098, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-4.345419288156761],[-2.9564760891365234]
4,Houthi Rebels representing Houthis that have b...,"The Saudi bombing campaign has been atrocious,...",1,10,"[-0.017771857, 0.11146448, -0.03416922, -0.047...","[0.00931892, 0.08899142, -0.0032000882, -0.053...","[0.8523809523805466, 0.028571428571414967, 0.0...","[0.8397435897430515, 0.03846153846151381, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09221979...",[-2.9564760891365234],[3.475064196855748]


In [ ]:
test_df.head()

,Paragraphs1,Paragraphs2,Truth_changes,file_number,Embeddings1,Embeddings2,Stylometrics1,Stylometrics2,Topics1,Topics2,Sentiments1,Sentiments2
0,"At least when you smear one second, you remain...",That's another way of saying that we all bette...,0,1,"[0.020681774, 0.011175946, 0.020733315, -0.045...","[0.04746337, 0.013456698, 0.08428604, -0.02112...","[0.748898678413767, 0.013215859030831183, 0.02...","[0.7813504823148614, 0.019292604501601515, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.03230306878685951, 0.03138899430632591...",[-4.64569098616402],[-4.652286126387811]
1,That's another way of saying that we all bette...,We already have very good synchronizing of clo...,0,1,"[0.04746337, 0.013456698, 0.08428604, -0.02112...","[-0.036173843, 0.022519007, 0.04480979, -0.019...","[0.7813504823148614, 0.019292604501601515, 0.0...","[0.7481662591685213, 0.029339853300726325, 0.0...","[0.0, 0.03230307251214981, 0.03138899058103561...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.079...",[-4.652286126387811],[-4.4985421187925]
2,We already have very good synchronizing of clo...,I think the earth's rotation and the year are ...,1,1,"[-0.036173843, 0.022519007, 0.04480979, -0.019...","[0.015000412, 0.013895365, 0.03997979, -0.0267...","[0.7481662591685213, 0.029339853300726325, 0.0...","[0.8079999999993536, 0.007999999999993599, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.079...","[0.0, 0.0, 0.0845579132437706, 0.0, 0.0, 0.0, ...",[-4.4985421187925],[43.828239106195426]
3,I think the earth's rotation and the year are ...,I would suggest that we replace months with de...,0,1,"[0.015000412, 0.013895365, 0.03997979, -0.0267...","[0.032227177, 0.039094165, 0.027847538, -0.038...","[0.8079999999993536, 0.007999999999993599, 0.0...","[0.8013698630131498, 0.00684931506848846, 0.01...","[0.0, 0.0, 0.08455739170312881, 0.0, 0.0, 0.0,...","[0.0, 0.08461221307516098, 0.08460406959056854...",[43.828239106195426],[-3.874107112522722]
4,"I think you mean ""euphemisms"" - when you basic...","The whole ""false dichotomy"" things makes no se...",0,10,"[0.039674997, -0.0010459255, 0.054714154, -0.0...","[-0.008312559, 0.008202695, -0.034693815, 0.01...","[0.7804878048777315, 0.008130081300809704, 0.0...","[0.7815699658700405, 0.006825938566550571, 0.0...","[0.0, 0.04412611201405525, 0.04415345937013626...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034...",[20.22492740867936],[-4.649030147250299]


In [ ]:
VALUE0 = 'Embeddings'
VALUE1 = 'Stylometrics'
VALUE2 = 'Sentiments'
VALUE3 = 'Topics'

pairs = [] # [ [ (value0, value1), (value0, value1), target {0, 1} ] [...] ]
pairs_only_embeddings = []
pairs_only_stylometrics = []
pairs_random = []
test_pairs = []

for index, row in df.iterrows():

    # Create tuples of values from pairs of columns along with the target value
    pair = [
        np.concatenate((row[f'{VALUE0}1'], row[f'{VALUE1}1'], row[f'{VALUE2}1'], row[f'{VALUE3}1'])),
        np.concatenate((row[f'{VALUE0}2'], row[f'{VALUE1}2'], row[f'{VALUE2}2'], row[f'{VALUE3}2'])),
        row['Truth_changes']
    ]
    pair_only_embeddings = [
        np.concatenate((row[f'{VALUE0}1'], row[f'{VALUE0}1'])),
        np.concatenate((row[f'{VALUE0}2'], row[f'{VALUE0}2'])),
        row['Truth_changes']
    ]
    pair_only_stylometrics = [
        np.concatenate((row[f'{VALUE1}1'], row[f'{VALUE1}1'])),
        np.concatenate((row[f'{VALUE1}2'], row[f'{VALUE1}2'])),
        row['Truth_changes']
    ]
    pair_only_topics = [
        np.concatenate((row[f'{VALUE3}1'], row[f'{VALUE3}1'])),
        np.concatenate((row[f'{VALUE3}2'], row[f'{VALUE3}2'])),
        row['Truth_changes']
    ]
    pair_sentiment_topics = [
        np.concatenate((row[f'{VALUE2}1'], row[f'{VALUE3}1'])),
        np.concatenate((row[f'{VALUE2}2'], row[f'{VALUE3}2'])),
        row['Truth_changes']
    ]
    pair_stylometric_embeddings = [
        np.concatenate((row[f'{VALUE0}1'], row[f'{VALUE1}1'])),
        np.concatenate((row[f'{VALUE0}2'], row[f'{VALUE1}2'])),
        row['Truth_changes']
    ]
    random_pair = [
        np.random.rand(868),
        np.random.rand(868),
        row['Truth_changes']
    ]

    pairs.append(pair)
    pairs_only_embeddings.append(pair_only_embeddings)
    pairs_only_stylometrics.append(pair_only_stylometrics)
    pairs_random.append(random_pair)


length = None

for index, row in test_df.iterrows():
    pair = [
        np.concatenate((row[f'{VALUE0}1'], row[f'{VALUE1}1'], row[f'{VALUE2}1'], row[f'{VALUE3}1'])),
        np.concatenate((row[f'{VALUE0}2'], row[f'{VALUE1}2'], row[f'{VALUE2}2'], row[f'{VALUE3}2'])),
        row['Truth_changes']
      ]
    test_pairs.append(pair)
    if length == None:
      length = len(pair[0])



In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adagrad, Adam, SGD
from torch.utils import data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

torch.set_grad_enabled(True)


In [ ]:
def data_loader(data):      # pairs

    final_data = {
        'embd_1' : torch.tensor,
        'embd_2' : torch.tensor,
        'gold' : int
    }

    prepared_data = []
    data_X = []
    labels = []

    for pair in data:
            final_data['tensor_1'] = torch.from_numpy(pair[0])
            final_data['tensor_2'] = torch.from_numpy(pair[1])
            final_data['gold'] = pair[2]

            stacked_tensor = torch.stack((final_data['tensor_1'], final_data['tensor_2']))
            stacked_tensor = stacked_tensor.reshape(1, 2, length)

            prepared_data.append((stacked_tensor, final_data['gold']))
            torch.stack((final_data['tensor_1'], final_data['tensor_2']))
            data_X.append(stacked_tensor)
            labels.append(final_data['gold'])

    print('Model input data prepared and processed..')

    return final_data, prepared_data, data_X, labels

In [ ]:
input_data, final_prepared_data, input_X, labels = data_loader(pairs)


Model input data prepared and processed..


In [ ]:
test_final_data, test_prepared_data, test_input_X, test_labels = data_loader(test_pairs)

Model input data prepared and processed..


In [ ]:
class SiameseFeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_dim2, output_dim, threshold=0.555):
        super(SiameseFeedForward, self).__init__()
        self.threshold = threshold
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.tan = nn.Tanh()
        self._initialize_weights()
        self.dropout = nn.Dropout(p=0.2)

    def _initialize_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, x, trace = None):

        in1 = x[:, 0, :]
        in2 = x[:, 1, :]

        out1 = self.fc1(in1)
        out2 = self.fc1(in2)
        # out1 = self.tan(out1)
        # out2 = self.tan(out2)
        out1 = self.dropout(out1)
        out2 = self.dropout(out2)
        # out1 = self.fc2(out1)
        # out2 = self.fc2(out2)
        # out1 = self.tan(out1)
        # out2 = self.tan(out2)
        # out1 = self.dropout(out1)
        # out2 = self.dropout(out2)
        # out1 = self.fc3(out1)
        # out2 = self.fc3(out2)


        if trace:
            print(out1.size(), out2.size())

        logits = F.cosine_similarity(out1, out2)

        preds = torch.sigmoid(logits)  # Apply sigmoid to convert to probabilities

        preds = 1 - (preds >= self.threshold).float()  # Convert probabilities to binary predictions

        return out1, out2, preds, logits


input_dim = length
hidden_dim = round(length/2)
hidden_dim2 = round(hidden_dim/2)
output_dim = round(hidden_dim2/2)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
import torch.nn.functional as F

class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=0.0, scale=5.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.scale = scale

    def forward(self, output1, output2, label):
        cos = F.cosine_similarity(output1, output2)
        cos = torch.sigmoid(cos)
        loss_contrastive = torch.mean(
            (1 - label) * torch.pow(1 - cos, 2) +                             # positive pairs, we want to pass how much they are similar
            (label) * torch.pow(torch.clamp(cos - self.margin, min=0.0), 2)   # negative pairs, we want to pass how much they are dissimilar
        )
        return loss_contrastive * self.scale

criterion = ContrastiveLoss()

# Cross-validate

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

def feed_to_tensor(tensor, data):
    for i, el in enumerate(data):
        tensor[i] = el

X_test, y_test = test_input_X, test_labels

final_tensor_test = torch.empty([len(X_test), 2, length])
feed_to_tensor(final_tensor_test, X_test)

labels_test = torch.empty(len(y_test))
feed_to_tensor(labels_test, y_test)

test = TensorDataset(final_tensor_test, labels_test)
final_test_loader = DataLoader(test,
                        batch_size = 64,
                         shuffle = False)

In [43]:
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.model_selection import StratifiedKFold, cross_val_score

skf = StratifiedKFold(n_splits=5)

skf.get_n_splits(input_X, labels)

best_score = np.inf
cv_scores = []
best_model = None

for i, (train_index, test_index) in enumerate(skf.split(input_X, labels)):

    model = SiameseFeedForward(input_dim, hidden_dim, hidden_dim2, output_dim)
    model = model.to(device)
    optimizer = Adam(model.parameters())

    print()
    print("=============")
    print(f"Fold {i}:")
    print("=============")
    print()

    X_train = [input_X[idx] for idx, data in zip(train_index, input_X)]
    X_val = [input_X[idx] for idx, data in zip(test_index, input_X)]

    y_train = [labels[idx] for idx, data in zip(train_index, labels)]
    y_val = [labels[idx] for idx, data in zip(test_index, labels)]

    final_tensor_train = torch.empty([len(X_train), 2, length])
    feed_to_tensor(final_tensor_train, X_train)

    labels_train = torch.empty(len(y_train))
    feed_to_tensor(labels_train, y_train)

    final_tensor_val = torch.empty([len(X_val), 2, length])
    feed_to_tensor(final_tensor_val, X_val)

    labels_val = torch.empty(len(y_val))
    feed_to_tensor(labels_val, y_val)

    class_counts = np.bincount(y_train)
    class_weights = 1. / class_counts
    weights = class_weights[y_train]
    print(weights)

    sampler = WeightedRandomSampler(weights, len(weights))

    train = TensorDataset(final_tensor_train, labels_train)
    train_loader = DataLoader(train,
                              batch_size = 64,
                              sampler = sampler)

    val = TensorDataset(final_tensor_val, labels_val)
    val_loader = DataLoader(val,
                            batch_size = 64,
                            shuffle = False)

    # ============ TRAIN MODEL ============


    for epoch in range(3):  #num_epochs
        print("epoch no", epoch)
        running_loss = 0.0
        for X, label in train_loader:

            X = X.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            #print(X.shape)
            #print(label.shape)

            out1, out2, preds, logits = model(X, trace=False)
            # print(out1, out2, sigm_logits)
            #print(scores.shape)
            #print(label.shape)

            # preds = torch.round(sigm_logits)
            loss = criterion(out1, out2, label)
            # print(preds, loss)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

            model.eval()

            with torch.no_grad():

                true_labels = []
                pred_labels = []

                for X_val, labels_val in val_loader:

                    X_val = X_val.to(device)
                    labels_val = labels_val.to(device)

                    _, _, preds, logits = model(X_val)

                    true_labels += labels_val.cpu().detach().numpy().tolist()
                    pred_labels += preds.cpu().detach().numpy().tolist()

        accuracy = accuracy_score(true_labels, pred_labels)
        cv_scores.append(accuracy)

        print(f'Epoch [{epoch+1}/{50}], Loss: {running_loss/len(train_loader)}')
        print(classification_report(true_labels, pred_labels))
        print("Accuracy", accuracy)
        print("Mean_accuracy", np.mean(cv_scores))
        if loss.item() < best_score:
          best_score = loss.item()
          best_model = model
          print("saved model with loss", best_score)




Fold 0:

[0.00014019 0.00014019 0.00014019 ... 0.00012378 0.00014019 0.00012378]
epoch no 0
Epoch [1/50], Loss: 1.379269620450605
              precision    recall  f1-score   support

         0.0       0.53      0.49      0.51      2019
         1.0       0.47      0.51      0.49      1784

    accuracy                           0.50      3803
   macro avg       0.50      0.50      0.50      3803
weighted avg       0.50      0.50      0.50      3803

Accuracy 0.49960557454641075
Mean_accuracy 0.49960557454641075
saved model with loss 1.166167974472046
epoch no 1
Epoch [2/50], Loss: 1.3119436527500634
              precision    recall  f1-score   support

         0.0       0.54      0.46      0.49      2019
         1.0       0.47      0.55      0.51      1784

    accuracy                           0.50      3803
   macro avg       0.50      0.50      0.50      3803
weighted avg       0.51      0.50      0.50      3803

Accuracy 0.500920326058375
Mean_accuracy 0.5002629503023929
ep

# Train & Test

In [ ]:
model = SiameseFeedForward(input_dim, hidden_dim, hidden_dim2, output_dim)
model = model.to(device)
optimizer = Adam(model.parameters(), lr=0.0005)

X_train, X_val, y_train, y_val = train_test_split(input_X, labels, train_size = 0.90)
X_test, y_test = test_input_X, test_labels

def feed_to_tensor(tensor, data):
    for i, el in enumerate(data):
        tensor[i] = el

final_tensor_train = torch.empty([len(X_train), 2, length])
feed_to_tensor(final_tensor_train, X_train)

labels_train = torch.empty(len(y_train))
feed_to_tensor(labels_train, y_train)

final_tensor_val = torch.empty([len(X_val), 2, length])
feed_to_tensor(final_tensor_val, X_val)

labels_val = torch.empty(len(y_val))
feed_to_tensor(labels_val, y_val)

final_tensor_test = torch.empty([len(X_test), 2, length])
feed_to_tensor(final_tensor_test, X_test)

labels_test = torch.empty(len(y_test))
feed_to_tensor(labels_test, y_test)

class_counts = np.bincount(y_train)
class_weights = 1. / class_counts
weights = class_weights[y_train]

sampler = WeightedRandomSampler(weights, len(weights))

train = TensorDataset(final_tensor_train, labels_train)
train_loader = DataLoader(train,
                          batch_size = 64,
                          sampler = sampler)

val = TensorDataset(final_tensor_val, labels_val)
val_loader = DataLoader(val,
                         batch_size = 64,
                        shuffle = False)

test = TensorDataset(final_tensor_test, labels_test)
test_loader = DataLoader(test,
                        batch_size = 64,
                         shuffle = False)

In [ ]:
best_model = None
best_score = np.inf
H = {"loss": [], "accuracy": []}

for epoch in range(100):  #num_epochs
        print("epoch no", epoch)
        running_loss = 0.0
        val_loss = 0.0
        for X, label in train_loader:

            X = X.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            #print(X.shape)
            #print(label.shape)

            out1, out2, preds, logits = model(X, trace=False)
            # print(out1, out2, sigm_logits)
            #print(scores.shape)
            #print(label.shape)

            # preds = torch.round(sigm_logits)
            loss = criterion(out1, out2, label)
            # print(preds, loss)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

            model.eval()

            with torch.no_grad():

                true_labels = []
                pred_labels = []

                for X_val, labels_val in val_loader:

                    X_val = X_val.to(device)
                    labels_val = labels_val.to(device)

                    out1, out2, preds, logits = model(X_val)

                    loss = criterion(out1, out2, labels_val)
                    val_loss += loss.item()

                    true_labels += labels_val.cpu().detach().numpy().tolist()
                    pred_labels += preds.cpu().detach().numpy().tolist()

        accuracy = accuracy_score(true_labels, pred_labels)
        H["loss"].append(running_loss / len(train_loader))
        H["accuracy"].append(val_loss / len(val_loader))
        print(f'Epoch [{epoch+1}/{50}], Loss: {running_loss/len(train_loader)}')
        print(classification_report(true_labels, pred_labels))
        print("Accuracy", accuracy)

        if val_loss / len(val_loader) < best_score:
          last_saved_epoch = epoch
          best_score = val_loss / len(val_loader)
          best_model = model
          print("saved model with loss", best_score)

        if epoch - last_saved_epoch >= 10:
          break

In [ ]:
# ============ Test MODEL ============

true_lab = []
pred_lab = []

for X, label in final_test_loader:
    X = X.to(device)
    label = label.to(device)

    _, _, preds, logits = best_model(X)

    true_lab += label.cpu().detach().numpy().tolist()
    pred_lab += preds.cpu().detach().numpy().tolist()

print(accuracy_score(true_lab, pred_lab))
print(confusion_matrix(true_lab, pred_lab))
print(classification_report(true_lab, pred_lab))

print(len(logits))

In [ ]:
import matplotlib.pyplot as plt

def plot_training(H):
    # Construct plots for loss and accuracy
    plt.style.use("ggplot")

    # Plot training loss
    plt.figure(figsize=(10, 5))
    plt.plot(H["loss"], label="train_loss")
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="upper right")
    plt.show()

    # Plot accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(H["accuracy"], label="accuracy", color="orange")
    plt.title("Validation Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(loc="lower right")
    plt.show()


plot_training(H)

In [ ]:
# PATH = os.path.join(INPUT_DIR, 'siamese-hard.pkl')

# torch.save(best_model.state_dict(), PATH)

In [ ]:
# PATH = os.path.join(INPUT_DIR, 'siamese-hard.pkl')

# new_model = SiameseFeedForward(input_dim, hidden_dim, hidden_dim2, output_dim)
# new_model = new_model.to(device)
# new_model.load_state_dict(torch.load(PATH))
# optimizer = Adam(new_model.parameters(), lr=0.0005)

In [ ]:
# ============ Test MODEL ============

true_lab = []
pred_lab = []
new_model.eval()
for X, label in final_test_loader:
    X = X.to(device)
    label = label.to(device)

    _, _, preds, logits = best_model(X)

    true_lab += label.cpu().detach().numpy().tolist()
    pred_lab += preds.cpu().detach().numpy().tolist()

print(accuracy_score(true_lab, pred_lab))
print(confusion_matrix(true_lab, pred_lab))
print(classification_report(true_lab, pred_lab))

print(len(logits))